## Set up directory

In [4]:
%cd /Users/ianleefmans/Desktop/Insight/Project/Re-Identifying_Persistent_Skin_Conditions/skinConditionDetect


/Users/ianleefmans/Desktop/Insight/Project/Re-Identifying_Persistent_Skin_Conditions/skinConditionDetect


# Example using custom datahelper and Faster-RCNN

In [5]:
# import packages
from datahelper import CreateDataset
import torch
from torch.utils.data import DataLoader
from PIL import Image
import os
import torchvision
#from model import MVP
from tqdm import tqdm
import os
import pandas as pd
import pickle

In [3]:
# Create paths
pickle_path = "/Users/ianleefmans/Desktop/Insight/Project/Re-Identifying_Persistent_Skin_Conditions/skinConditionDetect/annotation_dict.pkl"
data_directory = "/Users/ianleefmans/Desktop/Insight/Project/Data"

# Create dataset
dataset = CreateDataset(pickle_path, data_directory, local=True, transform = torchvision.transforms.ToTensor())
def my_collate(batch):
    data = [item[0] for item in batch]
    target = [item[1] for item in batch]
    #target = torch.LongTensor(target)
    return data, target
train_loader = DataLoader(dataset=dataset, batch_size=4, num_workers=0, shuffle=True, collate_fn=my_collate)
sample = iter(train_loader).next()


In [118]:
lis =[]
for i in tqdm(range(31085, 31130)):
    try:
        dataset[i]
    except:
        lis.append(i)
        print(i)
    finally:
        pass
lis

100%|██████████| 45/45 [00:04<00:00, 10.50it/s]


[]

In [81]:
len(dataset)

232333

In [ ]:
image = sample[0]
annotation = sample[1]

In [19]:
import pandas as pd
data_path = "/Users/ianleefmans/Desktop/Insight/Project/Data/followup_data/"

test_json = 'user_000000000277.json'

df = pd.read_json(data_path+test_json)



In [61]:
repeated_pics = []
for filename in tqdm(os.listdir("/Users/ianleefmans/Desktop/Insight/Project/Data/followup_data/")):
    df = pd.read_json("/Users/ianleefmans/Desktop/Insight/Project/Data/followup_data/"+filename)
    if "image_path" in df.columns:
        if df.image_path.nunique()==len(df)-df.image_path.isna().sum():
            pass
        else:
            vc = df.image_path.value_counts()
            repeated_pic+=list(vc[vc>1].index)
        

100%|██████████| 17293/17293 [04:06<00:00, 70.24it/s]


In [62]:
len(repeated_pics)

0

In [8]:
count = 0
for i, j in train_loader:
    print(type(i), type(j))
    count+=1
    if count==1:
        break

<class 'list'> <class 'list'>


In [11]:
type(dataset[44])

tuple

In [99]:
from tqdm import tqdm
import os
import pandas as pd
import pickle
annotation_dict = {}
count = 0 
tot_count=0
for filename in tqdm(os.listdir("/Users/ianleefmans/Desktop/Insight/Project/Data/followup_data/")):
    try:
        df = pd.read_json("/Users/ianleefmans/Desktop/Insight/Project/Data/followup_data/"+filename)
        df = df.loc[(df.image_details.notnull()) & (df.image_path.notnull()),:]
        for i in list(df.index.values):
            try:
                df.iloc[i].image_details['annotation'] !=None
                df.iloc[i]['image_details']['annotation'][0]
                ok = False
                for j in df.iloc[i].image_details['annotation']:
                    if (j['condition']=='Detected') and ('bounding_boxes' in j.keys()):
                        if len(j['bounding_boxes'])>0:
                            ok = True
                if ok ==False:
                    a = 2/0
                    
                annotation_dict[count] = (filename, i)
                
                count+=1
                
                    
            except:
                print
            finally:
                pass
            
    except:
        pass
    finally:
        pass
with open("annotation_dict.pkl", 'wb') as handle:
    pickle.dump(annotation_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

100%|██████████| 17293/17293 [07:27<00:00, 38.61it/s]


In [6]:
# open pickle
with open("annotation_dict.pkl", 'rb') as handle:
    dictionary = pickle.load(handle)

with open("bad_json_entry.pkl", 'rb') as handle:
    bad_json = pickle.load(handle)

In [ ]:
for i in bad_json:
    dictionary.pop(i)

newdictionary = {}
count= 0
for i in dictionary:
    newdictionary[count]=dictionary[i]
    count+=1

In [ ]:
with open("annotation_dict.pkl", 'wb') as handle:
    pickle.dump(newdictionary, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [44]:
len(dictionary) # dictionary length after removing 467 bad json

232333

In [9]:
len(dictionary) # dictionary length after removing 11 bad json

232800

In [79]:
len(dictionary) # dictionary length before removing 11 bad json


232811

In [19]:
len(dictionary)

232333

In [45]:
train_dictionary = {}
val_dictionary = {}
test_dictionary = {}
count1 = 0
count2 = 0
for i in tqdm(dictionary):
    if i<162632:
        train_dictionary[i] = dictionary[i]
    elif i<197483:
        val_dictionary[count1] = dictionary[i]
        count1+=1
    else:
        test_dictionary[count2] = dictionary[i]
        count2+=1

100%|██████████| 232333/232333 [00:00<00:00, 517815.67it/s]


In [46]:
with open("train_annotation_dict.pkl", 'wb') as handle:
    pickle.dump(train_dictionary, handle)

In [47]:
with open("val_annotation_dict.pkl", 'wb') as handle:
    pickle.dump(val_dictionary, handle)

In [48]:
with open("test_annotation_dict.pkl", 'wb') as handle:
    pickle.dump(test_dictionary, handle)